In [1]:
!pip install git+https://github.com/huggingface/transformers.git
!pip install datasets
!pip install huggingface-hub

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-i_8gnr_0
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-i_8gnr_0
  Resolved https://github.com/huggingface/transformers.git to commit 53e2fd785b2792e20f13189d30d1d4ef7d9cf673
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.5 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.33.0.dev0-py3-none-any.whl size=7633967 sha256=c61e98cb3fb3f42e5da187c53a700df88a0818fd4a7646cb3e26735df1ebfeaa
  Stored in directory: /tmp/pip-ephem-wheel-cache-lqq57h08/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16


In [2]:
import tensorflow as tf
from transformers import TFAutoModel ,AutoTokenizer, \
        pipeline, TFAutoModelForQuestionAnswering, TFTrainingArguments, TFTrainer
# import tensorflow_hub as hub
# import tensorflow_text as text
from PIL import Image
import os
from datasets import load_dataset,Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

### **Loading the data**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
train_data = pd.read_csv("data/train.csv")
test_data = pd.read_csv("data/test.csv")
# Selecting the relevant columns relevant to the task.
train_data = train_data[["review","question",'human_ans_spans',"human_ans_indices"]]
test_data = test_data[["review","question",'human_ans_spans',"human_ans_indices"]]

In [5]:
train_data.columns

Index(['review', 'question', 'human_ans_spans', 'human_ans_indices'], dtype='object')

### **EDA & Data Preproccessing**

In [6]:
train_cond = (train_data.human_ans_spans == "ANSWERNOTFOUND")
test_cond = (test_data.human_ans_spans == "ANSWERNOTFOUND")
print(train_data.loc[train_cond,["human_ans_spans"]].value_counts(),"\n")
print(test_data.loc[test_cond,["human_ans_spans"]].value_counts())

human_ans_spans
ANSWERNOTFOUND     1535
dtype: int64 

human_ans_spans
ANSWERNOTFOUND     382
dtype: int64


In [7]:
data_format_train = train_data.drop(train_data[train_cond].index,axis=0)
data_format_test = test_data.drop(test_data[test_cond].index,axis=0)

# cond_format = (data_format.human_ans_spans == "ANSWERNOTFOUND")

# NONE
# data_format.loc[cond_format,["human_ans_spans"]].value_counts()

In [8]:
# Train
data_format_train_reset_index = (data_format_train.reset_index(drop=True))
train_format_cond = (data_format_train_reset_index.
                            human_ans_spans == "ANSWERNOTFOUND")

(data_format_train_reset_index.
                loc[train_format_cond,
                            ["human_ans_spans"]].head(5))

,human_ans_spans


In [9]:
# Test
data_format_test_reset_index = (data_format_test.reset_index(drop=True))
test_format_cond = (data_format_test_reset_index.
                            human_ans_spans == "ANSWERNOTFOUND")

(data_format_test_reset_index.
                loc[test_format_cond,
                            ["human_ans_spans"]].head(5))

,human_ans_spans


In [10]:
data_format_train_reset_index[['human_ans_spans','human_ans_indices']].head()

,human_ans_spans,human_ans_indices
0,this show is OUTSTANDING,"(296, 320)"
1,The costume design by Susan Matheson is great,"(1254, 1299)"
2,Oh my god is this a STUPID film,"(1212, 1243)"
3,KingKong,"(9, 17)"
4,The acting is well done,"(1243, 1266)"


In [11]:
data_format_test_reset_index[['human_ans_spans','human_ans_indices']].head()

,human_ans_spans,human_ans_indices
0,just barely above mildly interesting,"(1850, 1886)"
1,important,"(240, 249)"
2,sound were crystal clear,"(111, 135)"
3,The only bright spot in the movie was that Jar...,"(705, 794)"
4,Ofelia 's father has been killed in the Spanis...,"(2026, 2078)"


In [12]:
data_format_train_reset_index.columns

Index(['review', 'question', 'human_ans_spans', 'human_ans_indices'], dtype='object')

In [13]:
data_format_train_reset_index.iloc[0].review

"At the time of my review, there had been 910 customer reviews.  Of these, there were 10 one-star, 10 two-star, 8 three-star, 34 four-star and 848 five-star reviews.  I know that you can't please everybody, but it's obvious how people feel about this show.  And I have to vote with the majority...this show is OUTSTANDING! ANSWERNOTFOUND"

In [14]:
data_format_train_reset_index.iloc[0].question

'Is this series good and excelent?'

In [15]:
data_format_train_reset_index.iloc[0].human_ans_spans

'this show is OUTSTANDING'

In [16]:
data_format_train_reset_index.iloc[0].human_ans_indices

'(296, 320)'

In [17]:
data_format_train_reset_index.iloc[0].review[366:709]

''

In [18]:
data_format_test_reset_index.columns

Index(['review', 'question', 'human_ans_spans', 'human_ans_indices'], dtype='object')

In [19]:
data_format_train_reset_index.columns

Index(['review', 'question', 'human_ans_spans', 'human_ans_indices'], dtype='object')

In [20]:
(
    int(data_format_train_reset_index.iloc[0].
human_ans_indices.
split('(')[1].
split(",")[0]))

296

In [21]:
def answer_normalise(df):
    x = df['human_ans_indices']
    answer = {}
    answer_start = x.split("(")[1].split(",")[0]
    answer['text'] = [df['human_ans_spans']]
    answer['answer_start'] = [int(answer_start)]
    return answer

In [22]:
data_format_train_reset_index['answers'] = (data_format_train_reset_index.
                                    apply(answer_normalise,axis=1))

In [23]:
data_format_test_reset_index['answers'] = (data_format_test_reset_index.
                                    apply(answer_normalise,axis=1))

In [24]:
# Train
data_format_train_reset_index = (data_format_train_reset_index.
drop(columns=['human_ans_spans',
                'human_ans_indices'],)
)

# Test
data_format_test_reset_index = (data_format_test_reset_index.
drop(columns=['human_ans_spans',
                'human_ans_indices'],
)
)

In [25]:
print(data_format_train_reset_index.columns)
print(data_format_test_reset_index.columns)

Index(['review', 'question', 'answers'], dtype='object')
Index(['review', 'question', 'answers'], dtype='object')


In [26]:
len(data_format_train_reset_index['answers'][0]['answer_start'])

1

In [27]:
train_dataset = data_format_train_reset_index
test_dataset = data_format_test_reset_index

In [28]:
print(train_dataset.columns)
print(test_dataset.columns)

Index(['review', 'question', 'answers'], dtype='object')
Index(['review', 'question', 'answers'], dtype='object')


In [29]:
train_dataset.iloc[0].review

"At the time of my review, there had been 910 customer reviews.  Of these, there were 10 one-star, 10 two-star, 8 three-star, 34 four-star and 848 five-star reviews.  I know that you can't please everybody, but it's obvious how people feel about this show.  And I have to vote with the majority...this show is OUTSTANDING! ANSWERNOTFOUND"

In [30]:
train_dataset.iloc[0].question

'Is this series good and excelent?'

In [31]:
train_dataset.iloc[0].answers['answer_start'][0]

296

### **Loading the dataset using HuggingFace datasets**

In [32]:
train_dataset.rename(columns={'review':'context'},inplace=True)
test_dataset.rename(columns={'review':'context'},inplace=True)

In [33]:
train_dataset.head()

,context,question,answers
0,"At the time of my review, there had been 910 c...",Is this series good and excelent?,"{'text': ['this show is OUTSTANDING'], 'answer..."
1,"""Fright Night"" is great! This is how the story...",How is the costume design?,{'text': ['The costume design by Susan Matheso...
2,Revenge of the Sith is for children and for ad...,What criticism deserves the movie Passion of C...,"{'text': ['Oh my god is this a STUPID film'], ..."
3,The best KingKong movie ever made in my opinio...,How is the scene?,"{'text': ['KingKong'], 'answer_start': [9]}"
4,"First of all, Prometheus is a prequel. It take...",How about the character of the actors in the m...,"{'text': ['The acting is well done'], 'answer_..."


In [34]:
original_trainset = Dataset.from_pandas(train_dataset)
original_test_data = Dataset.from_pandas(test_dataset)

In [35]:
print(original_trainset)
print(original_test_data)

Dataset({
    features: ['context', 'question', 'answers'],
    num_rows: 966
})
Dataset({
    features: ['context', 'question', 'answers'],
    num_rows: 200
})


### **Exploring the dataset**

In [36]:
print("Context: ", original_trainset["context"][0],"\n")
print("Question: ", original_trainset["question"][0],"\n")
print("Answer: ", original_trainset["answers"][0])

Context:  At the time of my review, there had been 910 customer reviews.  Of these, there were 10 one-star, 10 two-star, 8 three-star, 34 four-star and 848 five-star reviews.  I know that you can't please everybody, but it's obvious how people feel about this show.  And I have to vote with the majority...this show is OUTSTANDING! ANSWERNOTFOUND 

Question:  Is this series good and excelent? 

Answer:  {'answer_start': [296], 'text': ['this show is OUTSTANDING']}


### **Loading the model checkpoint**

In [37]:
model_checkpoint = "bert-base-cased"
tokeniser_checkpoint = "bert-base-cased"
model = TFAutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(tokeniser_checkpoint)

All PyTorch model weights were used when initializing TFBertForQuestionAnswering.

Some weights or buffers of the TF 2.0 model TFBertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
max_length = 384  # The maximum length of a feature (question and context)
doc_stride = (
    128  # The authorized overlap between two part of the context when splitting
)
# it is needed.

In [39]:
def prepare_features(examples):
    # Tokenize our examples with truncation and padding, but keep the overflows using a
    # stride. This results in one example possible giving several features when a context is long,
    # each of those features having a context that overlaps a bit the context of the previous
    # feature.
    examples["question"] = [q.lstrip() for q in examples["question"]]
    examples["context"] = [c.lstrip() for c in examples["context"]]
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a
    # map from a feature to its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original
    # context. This will help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what
        # is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this
        # span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the
            # CLS index).
            if not (
                offsets[token_start_index][0] <= start_char
                and offsets[token_end_index][1] >= end_char
            ):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the
                # answer.
                # Note: we could go after the last offset if the answer is the last word (edge
                # case).
                while (
                    token_start_index < len(offsets)
                    and offsets[token_start_index][0] <= start_char
                ):
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [40]:
train_dataset = original_trainset.map(
    prepare_features,
    batched=True,
    batch_size=10,
    remove_columns=original_trainset.column_names,
)
len(train_dataset), len(original_trainset)

Map:   0%|          | 0/966 [00:00<?, ? examples/s]

(1773, 966)

In [41]:
test_dataset = original_test_data.map(
    prepare_features,
    batched=True,
    batch_size=10,
    remove_columns=original_test_data.column_names,
)
len(test_dataset), len(original_test_data)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

(353, 200)

In [42]:
# Convert numpy format dataset to TensorFlow format
train_set = tf.data.Dataset.from_tensor_slices((
    {
        "input_ids": train_dataset["input_ids"],
        "attention_mask": train_dataset["attention_mask"]
    },
    {
        "start_positions": train_dataset["start_positions"],
        "end_positions": train_dataset["end_positions"]
    }
))


# Convert numpy format dataset to TensorFlow format
test_set = tf.data.Dataset.from_tensor_slices((
    {
        "input_ids": test_dataset["input_ids"],
        "attention_mask": test_dataset["attention_mask"]
    },
    {
        "start_positions": test_dataset["start_positions"],
        "end_positions": test_dataset["end_positions"]
    }
))

In [43]:
# Define the batch size
batch_size = 5 # Adjust this value as needed

In [44]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5,weight_decay=0.001)

In [45]:
# Optionally uncomment the next line for float16 training
tf.keras.mixed_precision.set_global_policy("mixed_float16")

model.compile(optimizer=optimizer)

The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


In [46]:
train_set = train_set.batch(batch_size)
test_set = test_set.batch(batch_size)

In [47]:
model.fit(train_set, validation_data=test_set, epochs=1)

355/355 [==============================] - 10631s 30s/step - loss: 2.8630 - val_loss: 2.5666


In [70]:
context1 = """The latest Spider-Man film is a
resounding success! Tom Holland shines as Peter Parker,
delivering a charismatic and emotionally resonant performance.
The action is thrilling, and the supporting cast adds humor and depth to the story.
This Spider-Man movie balances heart and spectacle brilliantly, making it a must-watch."""

question1 = "What did the supporting cast do?"

context2 = """The newest Spider-Man movie disappoints with its
focus on spectacle over substance. Tom Holland's character feels
one-dimensional, overshadowed by excessive CGI and action sequences.
The convoluted plot, numerous villains, and forced humor result in a
bloated and unfocused narrative. This film lacks the simplicity and
relatability that made Spider-Man iconic."""
question2 = "was it disappointing?"

In [71]:
inputs = tokenizer([context1], [question1], return_tensors="np")
outputs = model(inputs)
start_position = tf.argmax(outputs.start_logits, axis=1)
end_position = tf.argmax(outputs.end_logits, axis=1)
print(int(start_position), int(end_position[0]))

answer = inputs["input_ids"][0, int(start_position) : int(end_position) + 1]
print(answer)

print(tokenizer.decode(answer))

34 38
[ 1109  2168  1110 21401  1158]
The action is thrilling


In [72]:
inputs = tokenizer([context2], [question2], return_tensors="np")
outputs = model(inputs)
start_position = tf.argmax(outputs.start_logits, axis=1)
end_position = tf.argmax(outputs.end_logits, axis=1)
print(int(start_position), int(end_position[0]))

answer = inputs["input_ids"][0, int(start_position) : int(end_position) + 1]
print(answer)

print(tokenizer.decode(answer))

0 39
[  101  1109 15649  8454   118  2268  2523  4267  3202  8661 21506  1116
  1114  1157  2817  1113 25396  1166  9556   119  2545  7465   112   188
  1959  5115  1141   118  8611   117 17074  9574 12595  1118 12177   140
 21065  1105  2168 10028]
[CLS] The newest Spider - Man movie disappoints with its focus on spectacle over substance. Tom Holland's character feels one - dimensional, overshadowed by excessive CGI and action sequences


In [75]:
context = """Keras is an API designed for human beings, not machines. Keras follows best
practices for reducing cognitive load: it offers consistent & simple APIs, it minimizes
the number of user actions required for common use cases, and it provides clear &
actionable error messages. It also has extensive documentation and developer guides. """
question = "What is Keras for?"


In [76]:
inputs = tokenizer([context], [question], return_tensors="np")
outputs = model(inputs)
start_position = tf.argmax(outputs.start_logits, axis=1)
end_position = tf.argmax(outputs.end_logits, axis=1)
print(int(start_position), int(end_position[0]))

answer = inputs["input_ids"][0, int(start_position) : int(end_position) + 1]
print(answer)

print(tokenizer.decode(answer))

24 30
[ 1122  3272  8080   111  3014 20480  1116]
it offers consistent & simple APIs
